In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 数据集3：tax_info.csv
# 包含数据集7和8中涉及到的企业的纳税信息，每一行代表一个企业的纳税信息，每一行有9列，其中id列为企业唯一标识，列之间采用“,”分隔符分割。
# 数据格式如下：
# [id:企业唯一标识, START_DATE:起始时间, END_DATE:终止时间, TAX_CATEGORIES:税种, TAX_ITEMS:税目, TAXATION_BASIS:计税依据, TAX_RATE:税率, DEDUCTION:扣除数, TAX_AMOUNT:税额]
tax_info = pd.read_csv('../input/train/tax_info.csv')
print(tax_info.shape)
tax_info.info()

(29195, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29195 entries, 0 to 29194
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              29195 non-null  object 
 1   START_DATE      29195 non-null  object 
 2   END_DATE        29195 non-null  object 
 3   TAX_CATEGORIES  29195 non-null  object 
 4   TAX_ITEMS       29195 non-null  object 
 5   TAXATION_BASIS  25816 non-null  float64
 6   TAX_RATE        25816 non-null  float64
 7   DEDUCTION       24235 non-null  float64
 8   TAX_AMOUNT      29195 non-null  float64
dtypes: float64(4), object(5)
memory usage: 2.0+ MB


In [3]:
def identify_missing(df, missing_threshold):
    """
    缺失率
    @param df:
    @param missing_threshold:
    @return:
    """
    missing_rate = df.isnull().sum() / len(df)
    missing_rate = missing_rate.sort_values(ascending=False)
    print(missing_rate)
    to_drop = missing_rate[missing_rate > missing_threshold].index.to_list()
    print('{} features with greater than {} missing values.\n'.format(len(to_drop), missing_threshold))
    return to_drop

In [4]:
to_drop = identify_missing(tax_info, missing_threshold=0.5)
tax_info.drop(to_drop, axis=1, inplace=True)
to_drop

DEDUCTION         0.169892
TAX_RATE          0.115739
TAXATION_BASIS    0.115739
TAX_AMOUNT        0.000000
TAX_ITEMS         0.000000
TAX_CATEGORIES    0.000000
END_DATE          0.000000
START_DATE        0.000000
id                0.000000
dtype: float64
0 features with greater than 0.5 missing values.



[]